# SPARCED's Interactive Tutorial

Start here if you are new to the SPARCED model.
This tutorial covers both how to create and how to run a SPARCED model.

## (Optional) Place yourself into the correct folder

SPARCED is designed to be run from the ```SPARCED/src``` subfolder.
If you want to run SPARCED from elsewhere (like in this tutorial), you will first need to add a path to that folder.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../../SPARCED/src"))

# 0 - Imports

It is a good practice to add the imports at the beginning of your script or notebook.

In [2]:
# Step 1: model creation
from compilation.create_model import *
# Step 2: model simulation
from simulation.utils.ligands import *
from simulation.run_model import *

## 1 - Model creation

The model creation pipeline works as follow:
  1. Some TSV (tab separated values) **input files** are read to generate an Antimony model.
  2. Next, this **Antimony model** is converted into SBML format.
  3. Finally, the **SBML model** is compiled into an AMICI model
SPARCED can only run if an **AMICI model** was generated.

_NB: Although it is possible to run SPARCED starting from an Antimony or an SBML file, in this tutorial will only cover how to create a SPARCED model from the input files._

In [3]:
# Model name
model_name = "SPARCED"
# Path to input files written in TSV
f_compartments = "../../SPARCED/data/core/Compartments.txt"
f_observables = "../../SPARCED/data/core/Observables.txt"
f_species = "../../SPARCED/data/core/Species.txt"
f_stoichmatrix = "../../SPARCED/data/core/StoicMat.txt"
f_ratelaws = "../../SPARCED/data/core/Ratelaws.txt"
# Path to store output files (Antimony, SBML and AMICI model)
output_dir_path = "../../SPARCED/models/SPARCED-tutorial/" # Make sure that this directory exists ahead
f_output_parameters = "../../SPARCED/models/SPARCED-tutorial/out_Parameters.txt"
# Runtime options
verbose = True
is_SPARCED = True
# Call the model creation function
create_model(f_observables, model_name, f_compartments, f_stoichmatrix, output_dir_path,
             f_output_parameters, f_ratelaws, f_species, verbose, is_SPARCED)
# NB: AMICI is very verbose, don't be afraid if it loads tons of lines.
# It will take several minutes for the model to compile.

SPARCED: Success loading Antimony file
SPARCED: Success converting Antimony file to SBML


2024-04-29 12:11:48.994 - amici.sbml_import - INFO - Finished gathering local SBML symbols        + (8.89E+00s)
2024-04-29 12:11:49.205 - amici.sbml_import - INFO - Finished processing SBML parameters          + (2.01E-01s)
2024-04-29 12:11:49.213 - amici.sbml_import - INFO - Finished processing SBML compartments        + (6.71E-04s)
2024-04-29 12:11:50.657 - amici.sbml_import - INFO - Finished processing SBML species initials   ++ (1.36E+00s)
2024-04-29 12:11:50.665 - amici.sbml_import - INFO - Finished processing SBML rate rules         ++ (3.17E-05s)
2024-04-29 12:11:50.666 - amici.sbml_import - INFO - Finished processing SBML species             + (1.44E+00s)
2024-04-29 12:11:51.835 - amici.sbml_import - INFO - Finished processing SBML reactions           + (1.16E+00s)
2024-04-29 12:11:53.267 - amici.sbml_import - INFO - Finished processing SBML rules               + (1.42E+00s)
2024-04-29 12:11:53.275 - amici.sbml_import - INFO - Finished processing SBML initial assignments + (4.0

running build_ext
Changed extra_compile_args for unix to ['-fopenmp', '-std=c++14']
building 'SPARCED._SPARCED' extension
Testing SWIG executable swig4.0... SUCCEEDED.
swigging swig/SPARCED.i to swig/SPARCED_wrap.cpp
swig4.0 -python -c++ -modern -outdir SPARCED -I/home/chocolatecharlie/anaconda3/envs/sparced/lib/python3.12/site-packages/amici/swig -I/home/chocolatecharlie/anaconda3/envs/sparced/lib/python3.12/site-packages/amici/include -o swig/SPARCED_wrap.cpp swig/SPARCED.i
Deprecated command line option: -modern. This option is now always on.
gcc -pthread -B /home/chocolatecharlie/anaconda3/envs/sparced/compiler_compat -fno-strict-overflow -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /home/chocolatecharlie/anaconda3/envs/sparced/include -fPIC -O2 -isystem /home/chocolatecharlie/anaconda3/envs/sparced/include -fPIC -I/home/chocolatecharlie/Documents/SPARCED/SPARCED/models/SPARCED-tutorial/amici_SPARCED -I/home/chocolatecharlie/anaconda3/envs/sparced/lib/python3.12/site-packages/amici/includ

Although this might seem overwhelming, note that you may simply call the ```launch_model_creation``` function from your script or the command line, which will collect all the default values and arguments you pass, process these and call the ```create_model``` function for you.
We are not using it here to demonstrate which arguments the ```create_model``` function needs. A call to ```launch_model_creation``` looks as simple as this:

In [4]:
# WARNING: the Notebook doesn"t handle well command-line arguments
# See the __main__.py file for a working example
# launch_model_creation()

## 2 - Run a model simulation

### Ligands

You may want to add ligands to your experiment before running it (EGF and insulin are the most common).
There are several ways to do so, the idea being to chose the method that fits you best.

In [5]:
# 1. Manually
c_egf = 1.0
c_ins = 17.21
ligands = basic_ligands(c_egf, c_ins)
# 2. Automatically from the passed arguments
# See the __main__.py file for a working example
# ligands = basic_ligands_from_arguments
# 3. Automatically from the ligands.txt input file
# ligands = load_input_ligands("../../SPARCED/data/simulation/ligands.txt")

### Run an experiment

The ```run_experiment``` function is a wrapper that will call the ```run_single_simulation``` function for each cell in your population.

_Please note that in the example bellow, we redefined some variables previously set to give you an overview of what you need to call the ```run_experiment``` function. In practice, you wouldn't want to redefine variables in your script._

In [7]:
# Model and simulation names
model_name = "SPARCED"
simulation_name = "tutorial"
# Path to input files
f_species = "../../SPARCED/data/core/Species.txt"
f_genereg = "../../SPARCED/data/simulation/GeneReg.txt"
f_omics = "../../SPARCED/data/simulation/OmicsData.txt"
model_path = "../../SPARCED/models/SPARCED-tutorial/amici_SPARCED"
sbml_model = "../../SPARCED/models/SPARCED-tutorial/sbml_SPARCED.xml"
# Runtime options
is_deterministic = True # False implies stochastic simulation
exchange = 30 # Don't change this unless you are familiar with systems biology & models
population = 1 # Size of the cell population to simulation, one cell is enough here
duration = 1 # Duration of the simulated experiment, here 1 hour
verbose = True
is_SPARCED = True
# Additional arguments to run experiments involving drugs
# This part is not covered in this tutorial
compound = None
dose = None
# Call the run experimentfunction
run_experiment(model_name, model_path, simulation_name, sbml_model, is_deterministic, exchange,
               population, duration, f_species, ligands, f_genereg, f_omics, verbose, is_SPARCED, compound, dose)
# It will take several minutes for the model to run.

SPARCED: Successfully loaded <module 'SPARCED' from '/home/chocolatecharlie/Documents/SPARCED/SPARCED/models/SPARCED-tutorial/amici_SPARCED/SPARCED/__init__.py'>

tutorial n°0: Now ready to run

tutorial n°0: Simulation is over. Now saving results,                 please do not exit.

tutorial n°0: Simulation saved.



Although this might seem overwhelming, note that you may simply call the ```launch_experiment_simulation``` function from your script or the command line, which will collect all the default values and arguments you pass, process these and call the ```run_experiment``` function for you.
We are not using it here to demonstrate which arguments the ```run_experiment``` function needs. A call to ```launch_experiment_simulation``` looks as simple as this:

In [8]:
# WARNING: the Notebook doesn"t handle well command-line arguments
# See the __main__.py file for a working example
# launch_experiment_simulation(ligands)